<a href="https://colab.research.google.com/github/12234545/Atelier-4-c-poo-/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq langchain langchain-core langchain-groq chromadb pypdf gradio
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.0 MB/s eta 

In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.3 MB/s eta 0:00:00


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import LLMChain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage

import gradio as gr
import subprocess

In [ ]:
! curl -fsSL https://ollama.com/install.sh | sh
import subprocess
subprocess.Popen(["ollama", "serve"])
subprocess.run(["ollama", "pull", "nomic-embed-text"])

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


CompletedProcess(args=['ollama', 'pull', 'nomic-embed-text'], returncode=0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pdf_path = "/content/drive/MyDrive/PR/test2.pdf"
loader = PyPDFLoader(pdf_path)

In [ ]:
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

In [ ]:
!ollama pull deepseek-r1


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 96c415656d37...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 96c415656d37...   0% ▕▏ 630 KB/4.7 GB                  pulling manifest 
pulling 96c415656d37...   0% ▕▏ 9.7 MB/4.7 GB                  pulling manifest 
pulling 96c415656d37...   1% ▕▏  38 MB/4.7 GB                  pulling manifest 
pulling 96c415656d37...   1% ▕▏  50 MB/4.7 GB                  pulling manifest 
pulling 96c415656d37...   2% ▕▏  79 MB/4.7 GB                  pulling manifest 
pulling 96c415656d37...   2% ▕▏  95 MB/4.7 GB                  pulling manifest 
pulling 96c415656d37...   3% ▕▏ 122 MB/4.7 GB                  pulling manifest 
pulling 96c415656d37...   3% ▕▏ 140 MB/4.7 GB                  pulling manifest 
pulling 96c415656d37...   3% ▕▏ 162 MB/4.7 GB                  pulling manifest 
pulling 96c415656d37...   4% ▕▏ 181 MB/4.7 GB  171 MB/s     26spulling manifest 
pulling 96c4156

In [ ]:
subprocess.run(["ollama", "pull", "deepseek-r1"])

CompletedProcess(args=['ollama', 'pull', 'deepseek-r1'], returncode=0)

In [ ]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 970aa74c0a90... 100% ▕▏ 274 MB                         
pulling c71d239df917... 100% ▕▏  11 KB                         
pulling ce4a164fc046... 100% ▕▏   17 B                         
pulling 31df23ea7daa... 100% ▕▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
chunks

[Document(metadata={'producer': 'Microsoft® Excel®\xa02021', 'creator': 'Microsoft® Excel®\xa02021', 'creationdate': '2024-06-12T10:45:01+01:00', 'author': 'ERZINI', 'moddate': '2024-06-12T10:45:01+01:00', 'source': '/content/drive/MyDrive/PR/test2.pdf', 'total_pages': 14, 'page': 0, 'page_label': '1'}, page_content="1\nPour plus d’informations consulter les sites Web suivants : \n\uf0a8 Agence Marocaine de Coopération Internationale\xa0: www.amci.ma\n\uf0a8 Ministère de l’Enseignement Supérieur, de la Recherche Scientifique et de l'Innovation : www.enssup.gov.ma \n\uf0a8 Ministère de l’Education Nationale, du Préscolaire et des Sports : www.men.gov.ma\n\uf0a8 Office de la Formation Professionnelle et de la Promotion du Travail\xa0: www.ofppt.ma\nMars 2024\nANNEE ACADEMIQUE 2024/2025\nREPERTOIRE INDICATIF DES ETABLISSEMENTS PUBLICS\nMAROCAINS D'ENSEIGNEMENT SUPERIEUR"),
 Document(metadata={'producer': 'Microsoft® Excel®\xa02021', 'creator': 'Microsoft® Excel®\xa02021', 'creationdate': 

In [ ]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")

<ipython-input-12-90fd4e0c2d7c>:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")


In [ ]:
subprocess.Popen(["ollama", "serve"])

<Popen: returncode: None args: ['ollama', 'serve']>

In [ ]:
vectorstore = InMemoryVectorStore.from_texts(
    texts=[doc.page_content for doc in chunks],
    embedding=embeddings,
)

In [ ]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 8})

In [ ]:
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

llm_compression = ChatGroq(
    api_key="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",
    model="deepseek-llm-r1"
)

In [ ]:
compressor_llm = LLMChainExtractor.from_llm(llm_compression)
advanced_retriever = ContextualCompressionRetriever(
    base_compressor=compressor_llm,
    base_retriever=base_retriever
)

In [ ]:
def retrieve_context(query):
    """Récupère le contexte pertinent pour la question"""
    docs = advanced_retriever.get_relevant_documents(query)
    return "\n\n".join([f"- {doc.page_content}" for doc in docs])

In [ ]:
rag_template = """**Rôle** : Expert en enseignement supérieur au Maroc (en francais).
**Mission** : Fournir des informations exactes et à jour basées sur des rapports officiels.

**Historique** :
{history}

**Connaissances** :
{context}

**Question** :
{question}

**Consignes** :
- Répondez uniquement avec les informations disponibles
- En cas de hors-sujet : "Je n'ai pas cette information. Voulez-vous une question sur universités au marocaines ?"
- Ton professionnel et clair

**Format** :
Structure claire avec informations factuelles
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)


llm_response = ChatGroq(
    temperature=0,
    api_key="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",
    model="deepseek-llm-r1")


memory = ConversationBufferMemory(memory_key="history", return_messages=False)

def generate_response(user_message):
    """Génère la réponse de l'assistant"""

    conversation_history = memory.buffer


    context = retrieve_context(user_message)


    prompt = rag_prompt.format(
        history=conversation_history,
        context=context,
        question=user_message
    )


    response = llm_response([HumanMessage(content=prompt)]).content


    memory.save_context({"input": user_message}, {"output": response})

    return response

<ipython-input-20-2f524daeae4a>:31: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="history", return_messages=False)


In [ ]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## Consultant en education au Maroc")
    chatbot = gr.Chatbot()
    state = gr.State([])

    with gr.Row():
        txt_input = gr.Textbox(
            show_label=False,
            placeholder="Posez votre question ici...",
            container=False
        )
        send_btn = gr.Button("Envoyer")

    def user_interaction(user_message, history):
        response = generate_response(user_message)
        history.append((user_message, response))
        return "", history

    txt_input.submit(user_interaction, [txt_input, state], [txt_input, chatbot])
    send_btn.click(user_interaction, [txt_input, state], [txt_input, chatbot])

demo.launch()

<ipython-input-22-5d9cf1c91321>:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6151c7f99ca7fca759.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
